# NLP Bert Model

## Links:
- Bert Models: https://huggingface.co/ml6team/bert-base-uncased-city-country-ner
- Tuto site: https://www.projectpro.io/article/bert-nlp-model-explained/558

In [7]:
import torch
from transformers import BertForTokenClassification, BertTokenizer
from transformers import pipeline

In [ ]:
# Charger un modèle pré-entraîné BERT pour la classification de tokens (NER)
model_name = "ml6team/bert-base-uncased-city-country-ner" # dbmdz/bert-large-cased-finetuned-conll03-english
model = BertForTokenClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Définir les étiquettes pour les villes de départ et d'arrivée
labels = ["O", "B-START_CITY", "I-START_CITY", "B-DEST_CITY", "I-DEST_CITY"]

# Préparer les exemples d'entraînement (phrases annotées)
training_data = [
    ("Je veux prendre le train de Toulouse à Paris.", ["O", "O", "O", "B-START_CITY", "O", "B-DEST_CITY", "I-DEST_CITY", "O"]),
    # Ajoutez d'autres exemples ici
]

# Convertir les exemples en entrées du modèle
encoded_training_data = []
for text, labels in training_data:
    tokens = tokenizer(text, return_offsets_mapping=True)
    input_ids = tokens["input_ids"]
    attention_mask = tokens["attention_mask"]
    encoded_labels = [labels.index(label) for label in labels]
    encoded_training_data.append((input_ids, attention_mask, encoded_labels))

# Entraîner le modèle
train_dataset = torch.utils.data.TensorDataset(*[torch.tensor(t) for t in zip(*encoded_training_data)])

# Configurer l'entraînement
training_args = TrainingArguments(
    output_dir="./ner-model",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    evaluation_strategy="steps",
    save_steps=500,
    eval_steps=500,
)

# Créer un DataCollator pour les données d'entraînement
data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Lancer l'entraînement
trainer.train()

# Enregistrer le modèle entraîné
trainer.save_model("./ner-model")
